## 1. Importing Libraries

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Reading the Data

In [4]:
PROJECT_DIR = r"C:\Users\Aayush\Desktop\Flight Fare Prediction"
DATA_DIR = "data"

In [5]:
def retrieve_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [6]:
flights = retrieve_data("flight_price")

In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features
- Columns Route and Total_Stops have a missing value each
- The data types of some features are inappropriate

## 3. Preliminary Analysis

### 3.1 Check Data Types

In [8]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [10]:
value = flights.Date_of_Journey.iloc[20]

In [11]:
value

'6/03/2019'

In [12]:
value = flights.Dep_Time.iloc[20]
value

'14:10'

### 3.2 Check for Duplicates

In [13]:
flights.duplicated().sum()

220

In [14]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of `Date_of_Journey` and `Dep_Time` and `Arrival_Time` should be changed to datetime
- The type of `Duration` and `Total Stops` is mixed. It should be numeric type.
- There're 220 duplicates. These should be removed

## 4. Detailed Analysis

### Airline

In [15]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [16]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [17]:
(
    flights
    .Airline
    .str.replace(' Premium economy', '')
    .str.replace(' Business', '')
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

- some of the entries have inconsistent/inaccurate values

### Date_of_Journey

In [18]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [19]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [20]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [21]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_time

In [24]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [36]:
pd.to_datetime(flights.Dep_Time, format="%H:%M").dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [41]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [48]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str[-1]
    .unique()
    
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [72]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [67]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
    #.unique()
    
)

6474    5m
Name: Duration, dtype: object

In [73]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [98]:
(
    flights
    .drop(index=[6474])
    .Duration
    .str.split(" ", expand=True)
    .set_axis(['hour', 'minute'], axis=1)
    .assign(
        hour=lambda df_: (
            df_['hour'].
            str.replace('h', '')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace('m', '')
            .fillna('0')
            .astype(int)     
        )
    )
    .sum(axis=1)
    
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [105]:
(
    flights
    .drop(index=[6474])
    .Duration
    .str.split(" ", expand=True)
    .set_axis(['hour', 'minute'], axis=1)
    .assign(
        hour=lambda df_: (
            df_['hour'].
            str.replace('h', '')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace('m', '')
            .fillna('0')
            .astype(int)     
        )
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


- The duration for observation indexed 6474 is 5 minutes. This is clearly wrong. Will delete this observation

### Total_Stops

In [114]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [129]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
    .unique()
)

array([ 0.,  2.,  1.,  3., nan,  4.])

### Additional_Info

In [131]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [133]:
(
    flights
    .Additional_Info
    .replace("No info", "No Info")
    .unique()
)

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operations

In [110]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand=True)
        .set_axis(['hour', 'minute'], axis=1)
        .assign(
            hour=lambda df_: (
                df_['hour'].
                str.replace('h', '')
                .astype(int)
                .mul(60)
            ),
            minute = lambda df_: (
                df_
                .minute
                .str.replace('m', '')
                .fillna('0')
                .astype(int)     
            )
        )
        .sum(axis=1)
    )
    

In [145]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in  df.select_dtypes('O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline = lambda df_: (
                df_
                .airline
                .str.replace(' Premium economy', '')
                .str.replace(' Business', '')
                .str.title()
            )
        )
        .assign(
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time = lambda df_: pd.to_datetime(df_['dep_time'], format="%H:%M").dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_['arrival_time'], format='mixed').dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_: (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info = lambda df_: df_.additional_info.replace("No info", "No Info")
        )
        .drop(columns='route')
        
    )

In [146]:
flights_cleaned = clean_data(flights)
flights_cleaned
flights_cleaned.shape

(10462, 10)

## 6. Split the Data

In [147]:
flights_final = flights_cleaned.sample(1000)

In [148]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [149]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## 7. Export the Subsets

In [150]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    X.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).head()

In [151]:
export_data(X_train, y_train, 'train')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-27,Delhi,Cochin,14:00:00,19:00:00,300,1.0,In-flight meal not included,10262
1,Indigo,2019-03-27,Delhi,Cochin,06:40:00,22:20:00,940,1.0,No Info,6442
2,Jet Airways,2019-06-06,Kolkata,Banglore,14:05:00,04:40:00,875,1.0,No Info,13059
3,Jet Airways,2019-03-21,Banglore,New Delhi,05:45:00,16:10:00,625,1.0,In-flight meal not included,7832
4,Indigo,2019-06-12,Delhi,Cochin,18:35:00,01:30:00,415,1.0,No Info,5883


In [152]:
export_data(X_val, y_val, 'val')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-09,Kolkata,Banglore,06:30:00,19:50:00,800,1.0,No Info,14571
1,Jet Airways,2019-06-15,Delhi,Cochin,09:40:00,19:00:00,560,2.0,No Info,13014
2,Spicejet,2019-05-15,Banglore,Delhi,21:10:00,00:05:00,175,0.0,No Info,3625
3,Jet Airways,2019-05-15,Kolkata,Banglore,09:35:00,14:25:00,1730,1.0,No Info,11467
4,Multiple Carriers,2019-03-09,Delhi,Cochin,18:15:00,01:35:00,440,1.0,In-flight meal not included,14086


In [153]:
export_data(X_test, y_test, 'test')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-15,Kolkata,Banglore,16:30:00,22:35:00,365,1.0,In-flight meal not included,10844
1,Indigo,2019-06-09,Kolkata,Banglore,15:30:00,18:05:00,155,0.0,No Info,5224
2,Jet Airways,2019-05-15,Kolkata,Banglore,14:05:00,20:45:00,400,1.0,In-flight meal not included,10844
3,Vistara,2019-06-12,Banglore,Delhi,11:30:00,14:20:00,170,0.0,No Info,4668
4,Jet Airways,2019-05-21,Banglore,Delhi,19:50:00,22:50:00,180,0.0,In-flight meal not included,4030
